# Multiprocessing

## Tabla de contenidos
***



***

Aquí se revisará como se puede usar directamente múltiples threads o procesos para acelerar el código y que riesgos hay que tener en mente.

El módulo `threading` hace posible el correr código en paralelo en un único proceso. Esto hace el threading muy útil para tareas de I/O como leer/escribir sobre archivos o comunicación en redes, pero una opción inútil para cálculos pesados y lentos, donde el módulo `multiprocessing` brilla.

Con el módulo `multiprocessing`, se puede correr código en múltiples procesos, lo que significa que se puede correr código en múltiples cores de GPU, múltiples procesadores e incluso múltiples computadores.

El módulo `threading` es básico, en el sentido de que se tienen que crear y manejar los threads de forma manual. Para esto, se tiene el módulo `concurrent.futures`, que ofrece una manera simple de ejecutar una lista de tareas ya sea a través de threads o procesos.

## The Global Interpreter Lock (GIL)

El GIL es un bloqueo global (global lock) para el intérprete de Python, para que pueda ejecutar solo una instrucción a la vez. Un **lock** o **mutex (mutual exclusion)** en computación paralela es una sincronización primitiva que puede bloquear la ejecución paralela. Con un lock, se asegura que nadie puede tocar la variable mientras se está trabajando en ella.

Python ofrece diversas maneras de sincronización primitivas, somo `threading.Lock` y `threading.Semaphore`. Incluso con el módulo `threading`, solo se está ejecutando una sola instrucción a la vez en Python.

## El uso de múltiples threads

`threading` puede brindar muchos beneficios si se está esperando a recursos externos.

## ¿Por qué se necesita el GIL?

El GIL, es actualmente una parte esencial del intérprete de CPython porque se asegura de que el manejo de memoria es siempre consistente. Como el GIL se asegura de que una sola instrucción de Python se puede ejecutar simultáneamente, nunca hay problemas donde múltiples bits de código manipulan memoria la mismo tiempo, o donde memoria está siendo liberada al sistema que actualmente no está disponible.

## Múltiples threads y procesos

El módulo `multiprocessing` ha hecho bastante fácil el trabajar alrededor de las limitaciones del GIL porque cada proceso tiene su propio GIL.

El uso del módulo `multiprocessing` es bastante similar al del módulo `threading` pero tiene muchas características muy útiles que hacen mucho más sentido con múltiples procesos.

**IMPORTANTE: Debe ser consciente de que es crítico el poner en el código `if __name__ == '__main__'` cuando use `multiprocessing`. Cuando este módulo lanza los procesos extra de Python, va a ejecutar el mismo script de Python, así que sin este bloque de código usted va a terminar en un loop infinito de procesos que inician.**

## Ejemplos básicos

Para crear threads y prcoesos, se tienen diversas opciones:
- `concurrent.futures`: Una interfaz fácil de usar para correr funciones ya sea en threads o procesos, similar a `asyncio`.
- `threading`: Una interfaz para crear threads de forma directa.
- `multiprocessing`: Una interfaz con mucha utilidad y funciones convenientes para crear y manejar múltiples procesos de Python.

## concurrent.futures

In [1]:
import time
import concurrent.futures

def timer(name, steps, interval = 0.1):
    '''funcion timer que duerme steps * interval'''
    for step in range(steps):
        print(name, step)
        time.sleep(interval)
        

if __name__ == '__main__':
    #Reemplazar con concurrent.futures.ProcessPoolExecutor para
    #múltiples procesos en vez de threads
    
    with concurrent.futures.ThreadPoolExecutor() as executor:
        #Entregar la función a executor con algunos argumentos
        executor.submit(timer, steps = 3, name = 'a')
        
        #Dormir un poquito, para mantener el orden del output consistente
        time.sleep(0.1)
        executor.submit(timer, steps = 3, name = "b")

a 0
a 1
b 0
b 1
a 2
b 2


Primero se creó una función timer que corre time.sleep(interval) y lo hace steps veces. Antes de dormir, printea el nombre y el step actual así podemos ver fácilmente que es lo que está pasando. Luego, creamos executor usando `concurrent.futures.ThreadPoolExecutor` para ejecutar las funciones. Finalmente, entregamos las funciones que queremos ejecutar con sus respectivos argumentos para empezar ambos threads. Entre medio, dormimos por un pequeño intervalo de tiempo, así el output es consistente.

## threading

In [2]:
import time
import threading

def timer(name, steps, interval = 0.1):
    '''funcion timer que duerme steps * interval'''
    for step in range(steps):
        print(name, step)
        time.sleep(interval)
        
# Se crean los threads de forma declarativa
a = threading.Thread(target = timer, kwargs = dict(name = "a", steps = 3))
b = threading.Thread(target = timer, kwargs = dict(name = "b", steps = 3))

#Se empiezan los threads
a.start()

#Se duerme un poquito
time.sleep(0.1)
b.start()

a 0
a 1
b 0
ab 1
 2
b 2


La función timer es idéntica. En este caso creamos los threads instanciando `threading.Thread()` directamente, pero heredar de `threading.Thread` es también una opción. Los argumentos a la función objetivo pueden ser dados, pasando args/kwargs argumentos, pero estoy son opcionales si no se tiene necesidad de usarlos o si se han prellenado usando `functools.partial`.

Aquí estamos creando explícitamente los threads para correr una sola funcion y salir tan pronto como su tarea haya terminado. Esto es útil para threads que corren durante largos períodos, dado que este método requiere setear el thread para cada función.

In [4]:
import time
import threading

class Timer(threading.Thread):
    def __init__(self, name, steps, interval = 0.1):
        self.steps = steps
        self.interval = interval
        #threading.Thread tiene un nombre built- in
        #Be careful not to manually override it
        super().__init__(name = name)
        
        
    def run(self):
        '''funcion timer que duerme steps * interval'''
        for step in range(self.steps):
            print(self.name, step)
            time.sleep(self.interval)
a = Timer(name = "a", steps = 3)
b = Timer(name = "b", steps = 3)

a.start()

time.sleep(0.1)
b.start()

a 0
a 1
b 0
b 1
a 2
b 2


Diferencias críticas a tener en consideración: